# Data Retrieval From Feature Store

In [1]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
import hopsworks

In [2]:
project = hopsworks.login()

2025-08-30 11:03:06,598 INFO: Initializing external client
2025-08-30 11:03:06,599 INFO: Base URL: https://c.app.hopsworks.ai:443


2025-08-30 11:03:09,675 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1247380


In [3]:
fs = project.get_feature_store()

In [4]:
final_data = fs.get_feature_group('final_data', version=1)
query = final_data.select_all()
final_merge = query.read(read_options={"use_hive": True})

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (17.25s) 


In [5]:
final_merge.head()

,unique_id,truck_id,route_id,departure_date,estimated_arrival,delay,route_avg_temp,route_avg_wind_speed,route_avg_precip,route_avg_humidity,...,driver_id,name,gender,age,experience,driving_style,ratings,vehicle_no,average_speed_mph,is_midnight
0,3725,10497416,R-2c5432ed,2019-02-10 07:00:00+00:00,2019-02-12 16:54:00+00:00,1,28.818182,6.727273,0.0,51.272727,...,3fff1b6e-e,Brandon Cruz,male,53,23,proactive,6,10497416,59.77,1
1,4299,28978466,R-f8640cff,2019-01-19 07:00:00+00:00,2019-01-19 12:44:24+00:00,0,46.000000,9.000000,0.0,75.000000,...,c3362ffc-f,Brian Garcia,male,49,5,conservative,8,28978466,45.73,0
2,5674,13363106,R-07d2544a,2019-02-02 07:00:00+00:00,2019-02-04 03:04:48+00:00,0,49.888889,5.777778,0.0,67.444444,...,93e44ec9-5,Joel Howard,male,52,17,conservative,7,13363106,52.25,1
3,9783,61984883,R-d87e53cd,2019-02-09 07:00:00+00:00,2019-02-09 20:38:24+00:00,0,80.500000,10.000000,0.0,63.500000,...,42aa7479-5,Jerry Powers,male,41,7,conservative,2,61984883,56.94,0
4,4088,18855810,R-c061582f,2019-02-06 07:00:00+00:00,2019-02-06 19:22:48+00:00,0,22.000000,10.000000,0.0,73.500000,...,728ab9fc-f,John Vasquez,male,47,4,conservative,4,18855810,48.03,0


# Data Processing

In [6]:
final_merge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12308 entries, 0 to 12307
Data columns (total 49 columns):
 #   Column                          Non-Null Count  Dtype                  
---  ------                          --------------  -----                  
 0   unique_id                       12308 non-null  int64                  
 1   truck_id                        12308 non-null  int64                  
 2   route_id                        12308 non-null  object                 
 3   departure_date                  12308 non-null  datetime64[us, Etc/UTC]
 4   estimated_arrival               12308 non-null  datetime64[us, Etc/UTC]
 5   delay                           12308 non-null  int64                  
 6   route_avg_temp                  12308 non-null  float64                
 7   route_avg_wind_speed            12308 non-null  float64                
 8   route_avg_precip                12308 non-null  float64                
 9   route_avg_humidity              12308 n

In [7]:
final_merge.isna().sum()

unique_id                           0
truck_id                            0
route_id                            0
departure_date                      0
estimated_arrival                   0
delay                               0
route_avg_temp                      0
route_avg_wind_speed                0
route_avg_precip                    0
route_avg_humidity                  0
route_avg_visibility                0
route_avg_pressure                  0
route_description                   0
estimated_arrival_nearest_hour      0
departure_date_nearest_hour         0
origin_id                           0
destination_id                      0
distance                            0
average_hours                       0
origin_temp                         4
origin_wind_speed                   4
origin_description                  0
origin_precip                       4
origin_humidity                     4
origin_visibility                   4
origin_pressure                     4
destination_

In [8]:
final_merge[final_merge['origin_temp'].isnull()]

,unique_id,truck_id,route_id,departure_date,estimated_arrival,delay,route_avg_temp,route_avg_wind_speed,route_avg_precip,route_avg_humidity,...,driver_id,name,gender,age,experience,driving_style,ratings,vehicle_no,average_speed_mph,is_midnight
559,7661,18091756,R-112b790b,2019-01-25 07:00:00+00:00,2019-01-27 02:40:48+00:00,1,66.555556,6.888889,0.000000,90.888889,...,e975a383-c,Neil Herring,male,45,7,proactive,3,18091756,58.02,1
3782,8163,24746768,R-b5f9418a,2019-01-25 07:00:00+00:00,2019-01-27 14:35:24+00:00,0,47.454545,9.090909,0.000000,70.636364,...,3d91387f-2,William Anderson III,male,50,0,conservative,4,24746768,40.69,1
8379,11360,22916520,R-78ee1f97,2019-01-25 07:00:00+00:00,2019-01-28 10:08:24+00:00,0,57.500000,10.142857,0.000000,78.214286,...,ffedbf74-a,Thomas Ochoa,male,57,19,proactive,6,22916520,63.64,1
12077,7721,24654257,R-21472caf,2019-01-25 07:00:00+00:00,2019-01-27 16:50:24+00:00,0,69.000000,12.363636,0.018182,79.181818,...,f110642c-1,Marc Walters,male,47,5,proactive,3,24654257,61.93,1


In [9]:
final_merge[final_merge['origin_humidity'].isnull()]

,unique_id,truck_id,route_id,departure_date,estimated_arrival,delay,route_avg_temp,route_avg_wind_speed,route_avg_precip,route_avg_humidity,...,driver_id,name,gender,age,experience,driving_style,ratings,vehicle_no,average_speed_mph,is_midnight
559,7661,18091756,R-112b790b,2019-01-25 07:00:00+00:00,2019-01-27 02:40:48+00:00,1,66.555556,6.888889,0.000000,90.888889,...,e975a383-c,Neil Herring,male,45,7,proactive,3,18091756,58.02,1
3782,8163,24746768,R-b5f9418a,2019-01-25 07:00:00+00:00,2019-01-27 14:35:24+00:00,0,47.454545,9.090909,0.000000,70.636364,...,3d91387f-2,William Anderson III,male,50,0,conservative,4,24746768,40.69,1
8379,11360,22916520,R-78ee1f97,2019-01-25 07:00:00+00:00,2019-01-28 10:08:24+00:00,0,57.500000,10.142857,0.000000,78.214286,...,ffedbf74-a,Thomas Ochoa,male,57,19,proactive,6,22916520,63.64,1
12077,7721,24654257,R-21472caf,2019-01-25 07:00:00+00:00,2019-01-27 16:50:24+00:00,0,69.000000,12.363636,0.018182,79.181818,...,f110642c-1,Marc Walters,male,47,5,proactive,3,24654257,61.93,1


In [10]:
routes_data = fs.get_feature_group('routes_details_fg', version=1)

routes_data_query = routes_data.select_all()

routes_df = routes_data_query.read(read_options={"use_hive": True})

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (2.72s) 


In [11]:
routes_df[routes_df.route_id.isin(['R-112b790b', 'R-78ee1f97','R-b5f9418a', 'R-21472caf'])]

,route_id,origin_id,destination_id,distance,average_hours,event_time
30,R-78ee1f97,C-f8f01604,C-f5ed4c15,3757.02,75.14,2025-07-01 00:00:00+00:00
1460,R-21472caf,C-f8f01604,C-2e349ccd,2892.14,57.84,2025-07-01 00:00:00+00:00
1596,R-112b790b,C-f8f01604,C-d3bb431c,2183.94,43.68,2025-07-01 00:00:00+00:00
1725,R-b5f9418a,C-f8f01604,C-4fe0fa24,2779.33,55.59,2025-07-01 00:00:00+00:00


In [12]:
weather_data = fs.get_feature_group('city_weather_details_fg', version=1)

weather_query = weather_data.select_all()

weather_df = weather_query.read(read_options={"use_hive": True})

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (25.23s) 


In [15]:
weather_df[(weather_df.city_id=='C-f8f01604')]

,city_id,date,hour,temp,wind_speed,description,precip,humidity,visibility,pressure,chanceofrain,chanceoffog,chanceofsnow,chanceofthunder
173,C-f8f01604,2019-01-17 00:00:00+00:00,1200,82.0,11.0,Sunny,0.0,11,6.0,1015.0,0.0,0.0,0.0,0.0
204,C-f8f01604,2019-02-08 00:00:00+00:00,600,59.0,11.0,Sunny,0.0,13,6.0,1015.0,0.0,0.0,0.0,0.0
306,C-f8f01604,2019-01-30 00:00:00+00:00,2200,75.0,7.0,Partly cloudy,0.0,8,6.0,1014.0,0.0,0.0,0.0,0.0
343,C-f8f01604,2019-01-18 00:00:00+00:00,1800,75.0,11.0,Clear,0.0,13,6.0,1013.0,0.0,0.0,0.0,0.0
392,C-f8f01604,2019-02-07 00:00:00+00:00,1800,70.0,9.0,Clear,0.0,9,6.0,1013.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53905,C-f8f01604,2019-01-22 00:00:00+00:00,200,73.0,8.0,Clear,0.0,14,6.0,1012.0,0.0,0.0,0.0,0.0
53912,C-f8f01604,2019-01-17 00:00:00+00:00,1400,84.0,11.0,Sunny,0.0,10,6.0,1013.0,0.0,0.0,0.0,0.0
53918,C-f8f01604,2019-02-02 00:00:00+00:00,100,73.0,9.0,Partly cloudy,0.0,10,6.0,1012.0,0.0,0.0,0.0,0.0
54012,C-f8f01604,2019-01-31 00:00:00+00:00,1700,86.0,10.0,Partly cloudy,0.0,7,6.0,1012.0,0.0,0.0,0.0,0.0


In [13]:
weather_df[(weather_df.city_id=='C-f8f01604')&(weather_df.date==pd.to_datetime('2019-01-25'))]

,city_id,date,hour,temp,wind_speed,description,precip,humidity,visibility,pressure,chanceofrain,chanceoffog,chanceofsnow,chanceofthunder


In [14]:
final_merge=final_merge.dropna(subset =  ['origin_temp', 'origin_wind_speed', 'origin_precip',
                                'origin_humidity', 'origin_visibility', 'origin_pressure' ] ).reset_index(drop=True)

In [16]:
final_merge.isna().sum()

unique_id                           0
truck_id                            0
route_id                            0
departure_date                      0
estimated_arrival                   0
delay                               0
route_avg_temp                      0
route_avg_wind_speed                0
route_avg_precip                    0
route_avg_humidity                  0
route_avg_visibility                0
route_avg_pressure                  0
route_description                   0
estimated_arrival_nearest_hour      0
departure_date_nearest_hour         0
origin_id                           0
destination_id                      0
distance                            0
average_hours                       0
origin_temp                         0
origin_wind_speed                   0
origin_description                  0
origin_precip                       0
origin_humidity                     0
origin_visibility                   0
origin_pressure                     0
destination_

In [17]:
len(final_merge)

12304

# Train - Validation - Test Split

In [ ]:
cts_cols=['route_avg_temp', 'route_avg_wind_speed',
       'route_avg_precip', 'route_avg_humidity', 'route_avg_visibility',
       'route_avg_pressure', 'distance', 'average_hours',
       'origin_temp', 'origin_wind_speed', 'origin_precip', 'origin_humidity',
       'origin_visibility', 'origin_pressure',
       'destination_temp','destination_wind_speed','destination_precip',
       'destination_humidity', 'destination_visibility','destination_pressure',
        'avg_no_of_vehicles', 'truck_age','load_capacity_pounds', 'mileage_mpg',
        'age', 'experience','average_speed_mph']


cat_cols=['route_description',
       'origin_description', 'destination_description',
        'accident', 'fuel_type',
       'gender', 'driving_style', 'ratings','is_midnight']


target=['delay']

In [19]:
final_merge['estimated_arrival'].min(), final_merge['estimated_arrival'].max()

(Timestamp('2019-01-01 07:04:48+0000', tz='Etc/UTC'),
 Timestamp('2019-02-14 16:06:00+0000', tz='Etc/UTC'))

In [22]:
split1 = pd.to_datetime('2019-01-30', utc=True)
split2 = pd.to_datetime('2019-02-07', utc=True)

train_df = final_merge[final_merge['estimated_arrival'] <= split1]

validation_df = final_merge[
    (final_merge['estimated_arrival'] > split1) &
    (final_merge['estimated_arrival'] <= split2)
]

test_df = final_merge[final_merge['estimated_arrival'] > split2]


In [23]:
len(train_df),len(validation_df),len(test_df)

(8202, 2336, 1766)

In [ ]:
X_train=train_df[cts_cols+cat_cols]
y_train=train_df['delay']

In [25]:
X_valid = validation_df[cts_cols + cat_cols]
y_valid = validation_df['delay']

X_test=test_df[cts_cols+cat_cols]
y_test=test_df['delay']

In [26]:
load_capacity_mode = X_train['load_capacity_pounds'].mode()
load_capacity_mode

0    3000.0
Name: load_capacity_pounds, dtype: float64

In [27]:
X_train['load_capacity_pounds']=X_train['load_capacity_pounds'].fillna(load_capacity_mode.iloc[0])
X_valid['load_capacity_pounds']=X_valid['load_capacity_pounds'].fillna(load_capacity_mode.iloc[0])
X_test['load_capacity_pounds']=X_test['load_capacity_pounds'].fillna(load_capacity_mode.iloc[0])

## Encoding

In [29]:
from sklearn.preprocessing import OneHotEncoder
from pickle import dump

In [31]:
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

In [35]:
encode_columns = ['route_description', 'origin_description', 'destination_description', 'fuel_type', 'gender', 'driving_style']
encoder.fit(X_train[encode_columns])

,categories,'auto'
,drop,None
,sparse_output,False
,dtype,<class 'numpy.float64'>
,handle_unknown,'ignore'
,min_frequency,None
,max_categories,None
,feature_name_combiner,'concat'


In [36]:
encoded_features = list(encoder.get_feature_names_out(encode_columns))

In [37]:
encoded_features

['route_description_Blizzard',
 'route_description_Blowing snow',
 'route_description_Clear',
 'route_description_Cloudy',
 'route_description_Fog',
 'route_description_Freezing drizzle',
 'route_description_Freezing fog',
 'route_description_Heavy rain',
 'route_description_Heavy rain at times',
 'route_description_Heavy snow',
 'route_description_Light drizzle',
 'route_description_Light freezing rain',
 'route_description_Light rain',
 'route_description_Light rain shower',
 'route_description_Light sleet',
 'route_description_Light sleet showers',
 'route_description_Light snow',
 'route_description_Mist',
 'route_description_Moderate or heavy freezing rain',
 'route_description_Moderate or heavy rain shower',
 'route_description_Moderate or heavy rain with thunder',
 'route_description_Moderate or heavy sleet',
 'route_description_Moderate or heavy sleet showers',
 'route_description_Moderate or heavy snow showers',
 'route_description_Moderate or heavy snow with thunder',
 'route

In [38]:
X_train[encoded_features] = encoder.transform(X_train[encode_columns])

X_valid[encoded_features] = encoder.transform(X_valid[encode_columns])

X_test[encoded_features] = encoder.transform(X_test[encode_columns])

In [39]:
X_train = X_train.drop(encode_columns, axis=1)

X_valid = X_valid.drop(encode_columns, axis=1)

X_test = X_test.drop(encode_columns, axis=1)

In [40]:
dump(encoder, open('files/truck_data_encoder.pkl', 'wb'))

In [43]:
X_train[encoded_features].head(2)

,route_description_Blizzard,route_description_Blowing snow,route_description_Clear,route_description_Cloudy,route_description_Fog,route_description_Freezing drizzle,route_description_Freezing fog,route_description_Heavy rain,route_description_Heavy rain at times,route_description_Heavy snow,...,destination_description_Torrential rain shower,fuel_type_Unknown,fuel_type_diesel,fuel_type_gas,gender_Unknown,gender_female,gender_male,driving_style_Unknown,driving_style_conservative,driving_style_proactive
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0
5,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0


## Normalization

In [44]:
from sklearn.preprocessing import StandardScaler

In [45]:
scaler = StandardScaler()

In [46]:
X_train[cts_cols] = scaler.fit_transform(X_train[cts_cols])

In [47]:
X_valid[cts_cols] = scaler.transform(X_valid[cts_cols])
X_test[cts_cols] = scaler.transform(X_test[cts_cols])

In [48]:
dump(scaler, open('files/truck_data_scaler.pkl', 'wb'))

# Model Building and Experimentation

In [51]:
import wandb
import joblib
import os

In [52]:
wandb.login()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /home/thnhan301/.netrc
wandb: Currently logged in as: nhanthien-sv (nhanthien-sv-hcmute) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [53]:
PROJECT_NAME = 'truck-delay-classification'

In [55]:
from sklearn.metrics import f1_score, recall_score, confusion_matrix, roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV

In [56]:
comparison_columns = ['Model_Name', 'Train_F1score', 'Train_Recall', 'Valid_F1score', 'Valid_Recall', 'Test_F1score', 'Test_Recall']

comparison_df = pd.DataFrame()



def evaluate_models(model_name, model_defined_var, X_train, y_train, X_valid, y_valid, X_test, y_test):
  ''' This function predicts and evaluates various models for classification'''

  # train predictions
  y_train_pred = model_defined_var.predict(X_train)
  # train performance
  train_f1_score = f1_score(y_train, y_train_pred)
  train_recall = recall_score(y_train, y_train_pred)

  # validation predictions
  y_valid_pred = model_defined_var.predict(X_valid)
  # validation performance
  valid_f1_score = f1_score(y_valid, y_valid_pred)
  valid_recall = recall_score(y_valid, y_valid_pred)

  # test predictions
  y_pred = model_defined_var.predict(X_test)
  # test performance
  test_f1_score = f1_score(y_test, y_pred)
  test_recall = recall_score(y_test, y_pred)

  # Printing performance
  print("Train Results")
  print(f'F1 Score: {train_f1_score}')
  print(f'Recall Score: {train_recall}')
  print(f'Confusion Matrix: \n{confusion_matrix(y_train, y_train_pred)}')
  print(f'Area Under Curve: {roc_auc_score(y_train, y_train_pred)}')

  print(" ")

  print("Validation Results")
  print(f'F1 Score: {valid_f1_score}')
  print(f'Recall Score: {valid_recall}')
  print(f'Confusion Matrix: \n{confusion_matrix(y_valid, y_valid_pred)}')
  print(f'Area Under Curve: {roc_auc_score(y_valid, y_valid_pred)}')

  print(" ")

  print("Test Results")
  print(f'F1 Score: {test_f1_score}')
  print(f'Recall Score: {test_recall}')
  print(f'Confusion Matrix: \n{confusion_matrix(y_test, y_pred)}')
  print(f'Area Under Curve: {roc_auc_score(y_test, y_pred)}')

  # Saving our results
  global comparison_columns
  metric_scores = [model_name, train_f1_score, train_recall, valid_f1_score, valid_recall, test_f1_score, test_recall]
  final_dict = dict(zip(comparison_columns, metric_scores))
  return final_dict


final_list = []
def add_dic_to_final_df(final_dict):
  global final_list
  final_list.append(final_dict)
  global comparison_df
  comparison_df = pd.DataFrame(final_list, columns=comparison_columns)

### Class Weights

In [57]:
y_train.value_counts().to_dict()[1]

2651

In [58]:
weights = len(X_train)/(2*(y_train.value_counts().to_dict()[0])), len(X_train)/(2*(y_train.value_counts().to_dict()[1]))
weights

(0.7387858043595749, 1.5469634100339495)

In [59]:
log_reg = LogisticRegression(random_state=13, class_weight={0:weights[0], 1:weights[1]})
log_reg.fit(X_train,y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,"{0: 0.7387858043595749, 1: 1.5469634100339495}"
,random_state,13
,solver,'lbfgs'
,max_iter,100
,multi_class,'deprecated'


In [ ]:
logistic_results = evaluate_models("Logistic Regression", log_reg, X_train, y_train, X_valid, y_valid, X_test, y_test)
add_dic_to_final_df(logistic_results)

Train Results
F1 Score: 0.5748006379585326
Recall Score: 0.6797434930215013
Confusion Matrix: 
[[3734 1817]
 [ 849 1802]]
Area Under Curve: 0.6762075418629394
 
Validation Results
F1 Score: 0.6087408949011447
Recall Score: 0.6956004756242569
Confusion Matrix: 
[[999 496]
 [256 585]]
Area Under Curve: 0.6819139501867103
 
Test Results
F1 Score: 0.7251675807434491
Recall Score: 0.7428214731585518
Confusion Matrix: 
[[720 245]
 [206 595]]
Area Under Curve: 0.7444677313979288


In [61]:
comparison_df

,Model_Name,Train_F1score,Train_Recall,Valid_F1score,Valid_Recall,Test_F1score,Test_Recall
0,Logistic Regression,0.574801,0.679743,0.608741,0.6956,0.725168,0.742821


In [66]:
import joblib

CLASS_WEIGHTS = {0: weights[0], 1: weights[1]}


def train_logistic_model(
    X_train, y_train, X_valid, y_valid, X_test, y_test, project_name=PROJECT_NAME
):
    """Train logistic regression model and log results with Weights & Biases."""

    features = X_train.columns
    params = {"random_state": 13, "class_weight": CLASS_WEIGHTS}

    with wandb.init(project=project_name) as run:
        config = wandb.config
        config.update(params)  # log hyperparameters

        # Train model
        model = LogisticRegression(**params)
        model.fit(X_train, y_train)

        # Evaluate
        metrics = {
            "train_f1": f1_score(y_train, model.predict(X_train)),
            "valid_f1": f1_score(y_valid, model.predict(X_valid)),
            "test_f1": f1_score(y_test, model.predict(X_test)),
        }

        # Log metrics
        print(
            f"F1 scores → Train: {metrics['train_f1']:.4f}, "
            f"Valid: {metrics['valid_f1']:.4f}, "
            f"Test: {metrics['test_f1']:.4f}"
        )
        wandb.log(metrics)

        # Predictions for visualization
        y_preds = model.predict(X_test)
        y_probas = model.predict_proba(X_test)

        labels = ["No Delay", "Delay"]
        wandb.sklearn.plot_classifier(
            model,
            X_train, X_test, y_train, y_test,
            y_preds, y_probas,
            labels=labels,
            model_name="LogisticRegression",
            feature_names=features,
        )

        # Save and log model artifact
        model_path = "files/log-truck-model.pkl"
        joblib.dump(model, model_path)

        artifact = wandb.Artifact("LogisticRegression", type="model", metadata=dict(config))
        artifact.add_file(model_path)
        run.log_artifact(artifact)

    return model, metrics

In [67]:
train_logistic_model(X_train, y_train,X_valid, y_valid, X_test, y_test)

wandb: 
wandb: Plotting LogisticRegression.


F1 scores → Train: 0.5748, Valid: 0.6087, Test: 0.7252


wandb: Logged feature importances.
wandb: Logged confusion matrix.
wandb: Logged summary metrics.
wandb: Logged class proportions.
wandb: Logged calibration curve.
wandb: Logged roc curve.
wandb: Logged precision-recall curve.


test_f1,▁
train_f1,▁
valid_f1,▁
test_f1,0.72517
train_f1,0.5748
valid_f1,0.60874


(LogisticRegression(class_weight={0: 0.7387858043595749, 1: 1.5469634100339495},
                    random_state=13),
 {'train_f1': 0.5748006379585326,
  'valid_f1': 0.6087408949011447,
  'test_f1': 0.7251675807434491})

In [68]:
random_f = RandomForestClassifier(n_estimators=20, class_weight=CLASS_WEIGHTS, random_state=7)
random_f.fit(X_train, y_train)

randomf_results = evaluate_models("Random Forest", random_f,X_train, y_train, X_valid, y_valid, X_test, y_test)
add_dic_to_final_df(randomf_results)

Train Results
F1 Score: 0.9922068047899639
Recall Score: 0.984534138061109
Confusion Matrix: 
[[5551    0]
 [  41 2610]]
Area Under Curve: 0.9922670690305545
 
Validation Results
F1 Score: 0.5148833467417538
Recall Score: 0.380499405469679
Confusion Matrix: 
[[1413   82]
 [ 521  320]]
Area Under Curve: 0.6628249535709599
 
Test Results
F1 Score: 0.6226870474658085
Recall Score: 0.48314606741573035
Confusion Matrix: 
[[910  55]
 [414 387]]
Area Under Curve: 0.7130756243814403


In [69]:
comparison_df

,Model_Name,Train_F1score,Train_Recall,Valid_F1score,Valid_Recall,Test_F1score,Test_Recall
0,Logistic Regression,0.574801,0.679743,0.608741,0.695600,0.725168,0.742821
1,Random Forest,0.992207,0.984534,0.514883,0.380499,0.622687,0.483146


In [70]:
def train_random_forest(
    X_train, y_train, X_valid, y_valid, X_test, y_test,
    project_name=PROJECT_NAME,
    model_params=None,
    model_path="files/randomf-truck-model.pkl"
):
    """
    Train a Random Forest model, evaluate on train/valid/test,
    log results & artifacts to Weights & Biases.
    """
    default_params = {
        "n_estimators": 100,
        "class_weight": CLASS_WEIGHTS,
        "random_state": 7
    }
    if model_params:
        default_params.update(model_params)

    features = X_train.columns

    with wandb.init(project=project_name) as run:
        config = wandb.config
        config.update(default_params)  # log hyperparams

        # Train model
        model = RandomForestClassifier(**default_params)
        model.fit(X_train, y_train)

        # Evaluate
        metrics = {
            "train_f1": f1_score(y_train, model.predict(X_train)),
            "valid_f1": f1_score(y_valid, model.predict(X_valid)),
            "test_f1":  f1_score(y_test,  model.predict(X_test)),
        }

        print(
            f"F1 scores → Train: {metrics['train_f1']:.4f}, "
            f"Valid: {metrics['valid_f1']:.4f}, "
            f"Test: {metrics['test_f1']:.4f}"
        )
        wandb.log(metrics)

        # Predictions for visualization
        y_preds = model.predict(X_test)
        y_probas = model.predict_proba(X_test)

        labels = ["No Delay", "Delay"]
        wandb.sklearn.plot_classifier(
            model,
            X_train, X_test, y_train, y_test,
            y_preds, y_probas,
            labels=labels, 
            model_name="RandomForestClassifier",
            feature_names=features,
        )

        # Save and log model artifact
        joblib.dump(model, model_path)
        artifact = wandb.Artifact("RandomForestClassifier", type="model", metadata=dict(config))
        artifact.add_file(model_path)
        run.log_artifact(artifact)

    return model, metrics

In [71]:
train_random_forest(X_train, y_train,X_valid, y_valid, X_test, y_test)

wandb: 
wandb: Plotting RandomForestClassifier.


F1 scores → Train: 1.0000, Valid: 0.5816, Test: 0.6651


wandb: Logged feature importances.
wandb: Logged confusion matrix.
wandb: Logged summary metrics.
wandb: Logged class proportions.
wandb: Logged calibration curve.
wandb: Logged roc curve.
wandb: Logged precision-recall curve.


test_f1,▁
train_f1,▁
valid_f1,▁
test_f1,0.66512
train_f1,1
valid_f1,0.58165


(RandomForestClassifier(class_weight={0: 0.7387858043595749,
                                      1: 1.5469634100339495},
                        random_state=7),
 {'train_f1': 1.0,
  'valid_f1': 0.5816485225505443,
  'test_f1': 0.6651198762567672})

In [72]:
import xgboost as xgb

dtrain = xgb.DMatrix(X_train, label=y_train)
dvalid = xgb.DMatrix(X_valid, label=y_valid)
dtest = xgb.DMatrix(X_test, label=y_test)

# Train initial model
params = {'objective': 'multi:softmax', 'num_class': 2, 'seed': 7}
num_rounds = 30
xgbmodel = xgb.train(params, dtrain, num_rounds, evals=[(dvalid, 'validation')], early_stopping_rounds=10)

xgb_results = evaluate_models("XGB", xgbmodel, dtrain, y_train, dvalid, y_valid, dtest, y_test)
add_dic_to_final_df(xgb_results)

[0]	validation-mlogloss:0.61450
[1]	validation-mlogloss:0.57317
[2]	validation-mlogloss:0.54769
[3]	validation-mlogloss:0.53642
[4]	validation-mlogloss:0.53487
[5]	validation-mlogloss:0.53000
[6]	validation-mlogloss:0.53028
[7]	validation-mlogloss:0.53020
[8]	validation-mlogloss:0.53448
[9]	validation-mlogloss:0.53441
[10]	validation-mlogloss:0.53978
[11]	validation-mlogloss:0.54023
[12]	validation-mlogloss:0.54750
[13]	validation-mlogloss:0.55097
[14]	validation-mlogloss:0.55124
[15]	validation-mlogloss:0.54851
Train Results
F1 Score: 0.7154022988505747
Recall Score: 0.5869483213881554
Confusion Matrix: 
[[5408  143]
 [1095 1556]]
Area Under Curve: 0.7805935986331878
 
Validation Results
F1 Score: 0.5898004434589801
Recall Score: 0.47443519619500596
Confusion Matrix: 
[[1382  113]
 [ 442  399]]
Area Under Curve: 0.6994249559570347
 
Test Results
F1 Score: 0.691285081240768
Recall Score: 0.5842696629213483
Confusion Matrix: 
[[880  85]
 [333 468]]
Area Under Curve: 0.7480933806834721


In [73]:
comparison_df

,Model_Name,Train_F1score,Train_Recall,Valid_F1score,Valid_Recall,Test_F1score,Test_Recall
0,Logistic Regression,0.574801,0.679743,0.608741,0.695600,0.725168,0.742821
1,Random Forest,0.992207,0.984534,0.514883,0.380499,0.622687,0.483146
2,XGB,0.715402,0.586948,0.589800,0.474435,0.691285,0.584270


In [74]:
import joblib
import xgboost as xgb
from sklearn.metrics import f1_score
import wandb


def train_xgb_model(
    X_train, y_train, X_valid, y_valid, X_test, y_test,
    project_name=PROJECT_NAME,
    model_params=None,
    num_rounds=100,
    early_stopping=10,
    model_path="files/xgb-truck-model.pkl"
):
    """
    Train an XGBoost model, evaluate on train/valid/test,
    log results & artifacts to Weights & Biases.
    """

    default_params = {
        "objective": "multi:softmax",
        "num_class": 2,
        "eval_metric": "mlogloss"
    }
    if model_params:
        default_params.update(model_params)

    # Convert to DMatrix
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dvalid = xgb.DMatrix(X_valid, label=y_valid)
    dtest  = xgb.DMatrix(X_test,  label=y_test)

    with wandb.init(project=project_name) as run:
        config = wandb.config
        config.update(default_params)

        # Train model with early stopping
        evals = [(dtrain, "train"), (dvalid, "valid")]
        model = xgb.train(
            default_params,
            dtrain,
            num_boost_round=num_rounds,
            evals=evals,
            early_stopping_rounds=early_stopping,
            verbose_eval=False
        )

        # Evaluate
        metrics = {
            "train_f1": f1_score(y_train, model.predict(dtrain)),
            "valid_f1": f1_score(y_valid, model.predict(dvalid)),
            "test_f1":  f1_score(y_test,  model.predict(dtest)),
        }

        print(
            f"F1 scores → Train: {metrics['train_f1']:.4f}, "
            f"Valid: {metrics['valid_f1']:.4f}, "
            f"Test: {metrics['test_f1']:.4f}"
        )
        wandb.log(metrics)

        # Save and log model artifact
        joblib.dump(model, model_path)
        artifact = wandb.Artifact("XGBoost", type="model", metadata=dict(config))
        artifact.add_file(model_path)
        run.log_artifact(artifact)

    return model, metrics

In [75]:
train_xgb_model(X_train, y_train,X_valid, y_valid, X_test, y_test)

F1 scores → Train: 0.7154, Valid: 0.5898, Test: 0.6913


test_f1,▁
train_f1,▁
valid_f1,▁
test_f1,0.69129
train_f1,0.7154
valid_f1,0.5898


(<xgboost.core.Booster at 0x70143861ff10>,
 {'train_f1': 0.7154022988505747,
  'valid_f1': 0.5898004434589801,
  'test_f1': 0.691285081240768})

In [76]:
import joblib
w = {0: weights[0], 1: weights[1]}
def train_rf_model(X_train=X_train, y_train=y_train, X_valid=X_valid,y_valid=y_valid, X_test=X_test, y_test=y_test):
    features = X_train.columns

    with wandb.init(
        project=PROJECT_NAME ) as run:
        config = wandb.config

        model = RandomForestClassifier(
            n_estimators=config["n_estimators"],
            max_depth=config["max_depth"],
            min_samples_split=config["min_samples_split"],
            random_state=7,
            class_weight=w
        )
        model.fit(X_train, y_train)
        
        # train predictions
        y_train_pred = model.predict(X_train)
        # train performance
        train_f1_score = f1_score(y_train, y_train_pred)
        

        # validation predictions
        y_valid_pred = model.predict(X_valid)
        # validation performance
        valid_f1_score = f1_score(y_valid, y_valid_pred)
      

        y_preds = model.predict(X_test)
        y_probas = model.predict_proba(X_test)

        score = f1_score(y_test, y_preds)
        print(f"F1_score Train: {round(train_f1_score, 4)}")
        print(f"F1_score Valid: {round(valid_f1_score, 4)}")
        print(f"F1_score Test: {round(score, 4)}")


        wandb.log({"f1_score_train": train_f1_score})
        wandb.log({"f1_score_valid": valid_f1_score})
        wandb.log({"f1_score": score})

        wandb.sklearn.plot_classifier(model, X_train, X_test, y_train, y_test, y_preds, y_probas, labels=None,
                                                          model_name='RandomForestClassifier', feature_names=features)

        model_artifact = wandb.Artifact(
            "RandomForestClassifier", type="model",metadata=dict(config))
        joblib.dump(model, "files/random_f_tuned.pkl")
        model_artifact.add_file("files/random_f_tuned.pkl")
        run.log_artifact(model_artifact)

In [ ]:
sweep_configs = {
    "method": "grid",
    "metric": {
        "name": "f1_score",
        "goal": "maximize"
    },
    "parameters": {
        "n_estimators": {
            "values": [8, 12, 16,20]
        },
        "max_depth": {
            "values": [None, 5, 10, 15, 20]
        },
        "min_samples_split": {
            "values": [2, 4, 8, 12]
        }
    }
}

sweep_id = wandb.sweep(
    sweep=sweep_configs,
    project=PROJECT_NAME
)



wandb.agent(
    project=PROJECT_NAME,
    sweep_id=sweep_id,
    function=train_rf_model
)

Create sweep with ID: ze0902xf
Sweep URL: https://wandb.ai/nhanthien-sv-hcmute/truck-delay-classification/sweeps/ze0902xf


wandb: Agent Starting Run: i1n9mlk9 with config:
wandb: 	max_depth: None
wandb: 	min_samples_split: 2
wandb: 	n_estimators: 8


wandb: 
wandb: Plotting RandomForestClassifier.


F1_score Train: 0.9554
F1_score Valid: 0.4588
F1_score Test: 0.5923


wandb: Logged feature importances.
wandb: Logged confusion matrix.
wandb: Logged summary metrics.
wandb: Logged class proportions.
wandb: Logged calibration curve.
wandb: Logged roc curve.
wandb: Logged precision-recall curve.


...

wandb: 
wandb: Plotting RandomForestClassifier.


F1_score Train: 0.5806
F1_score Valid: 0.6571
F1_score Test: 0.7633


wandb: Logged feature importances.
wandb: Logged confusion matrix.
wandb: Logged summary metrics.
wandb: Logged class proportions.
wandb: Logged calibration curve.
wandb: Logged roc curve.
wandb: Logged precision-recall curve.


wandb: Ctrl + C detected. Stopping sweep.


wandb: ERROR Problem finishing run
Traceback (most recent call last):
  File "/home/thnhan301/anaconda3/envs/llm/lib/python3.10/site-packages/wandb/agents/pyagent.py", line 297, in _run_job
    self._function()
  File "/tmp/ipykernel_11687/92056521.py", line 6, in train_rf_model
    with wandb.init(
  File "/home/thnhan301/anaconda3/envs/llm/lib/python3.10/site-packages/wandb/sdk/wandb_run.py", line 3626, in __exit__
    self._finish(exit_code=exit_code)
  File "/home/thnhan301/anaconda3/envs/llm/lib/python3.10/site-packages/wandb/sdk/wandb_run.py", line 399, in wrapper
    return func(self, *args, **kwargs)
  File "/home/thnhan301/anaconda3/envs/llm/lib/python3.10/site-packages/wandb/sdk/wandb_run.py", line 2276, in _finish
    self._atexit_cleanup(exit_code=exit_code)
  File "/home/thnhan301/anaconda3/envs/llm/lib/python3.10/site-packages/wandb/sdk/wandb_run.py", line 2471, in _atexit_cleanup
    self._on_finish()
  File "/home/thnhan301/anaconda3/envs/llm/lib/python3.10/site-package